# **Week 1**

#### **Objective: Evaluate the Data Set using Polynomial and Interaction Terms**

Each dataset has already been explored through univariate, bivariate, and multivariate analysis, including assissments of variable interactions and correlations with the target outcome. 

in this section, we extend that analysis by explicitly evaluating polynomial transformations and interaction terms (both categorical and numeric). The goal is to capture potential nonlinear relationships and combined effects between variables, and to assess how these features contribute to prediction the target. 

___

#### **Package Imports**

In [83]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# VIF Imports
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder



pd.reset_option('all')

/var/folders/wg/hrk6rd4s2mv7ztvpxryxn48c0000gn/T/ipykernel_2601/2384015227.py:15: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/var/folders/wg/hrk6rd4s2mv7ztvpxryxn48c0000gn/T/ipykernel_2601/2384015227.py:15: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


#### **Import Datasets**

In [4]:
BDB_All_Plays = pd.read_csv("../../AFL_Final_Project/BDB_All_Plays.csv") # Big Data Bowl Dataset
FNF_All_Plays = pd.read_csv("../../AFL_Final_Project/FNF_All_Plays.csv") # First and Future Dataset
PDA_All_Plays = pd.read_csv('../../AFL_Final_Project/PDA_All_Plays.csv') # Punt Data Analytics

#### **Important Functions**

In [5]:
# =======================================================
# Taken from Module 3 Milestone 1
# # Link: https://github.com/LeeMcFarling/Module_3_Milestone_1/blob/main/Milestone_01.ipynb
# 
# Split df into Numeric and Categorical Datasets, so that 
# visualizations can be catered accordingly. 
# =======================================================

# Numeric Columns
def numerify(df):
    numeric_cols = df.select_dtypes(include='number').columns
    filtered_cols = [col for col in numeric_cols if 'id' not in col.lower()] # Filter OUT 'id' columns
    df_numeric = df[filtered_cols]
    return df_numeric

# Categorical Columns
def categorify(df):
    pot_id_cols = ('gameId','playId','nflId','playerId','teamId','stadiumId')
    valid_id_columns = [c for c in pot_id_cols if c in df.columns]

    df_categorical_cols = df.select_dtypes(exclude=['number']).columns.tolist()

    combined_cat_cols = df_categorical_cols + valid_id_columns
    df_categorical = df[combined_cat_cols]
    return df_categorical



In [6]:
# ========================================================================================
# Taken from Module 3 - Final Project Milestone 1
# Link: https://github.com/LeeMcFarling/Module_3_Milestone_1/blob/main/Milestone_01.ipynb
# 
# Prupose:
# This is meant to consolidate the 'show_null_counts_features' function from before with 
# another with 'value' and 'unique' counts later on in this analysis. 
# ========================================================================================

def profile_dataset(df):
    # Identify feature types
    feature_types = df.dtypes.apply(lambda x: 'Numeric' if np.issubdtype(x, np.number) else 'Categorical')

    # Build a summary DataFrame
    summary = pd.DataFrame({
        'Feature': df.columns,
        'Type': feature_types.values,
        'Null Values': df.isnull().sum().values,
        'Null %': (df.isnull().mean() * 100).round(2).values,
        'Count (Non-Null)': df.count().values,
        'Unique Values': df.nunique().values
    })

    # Sort Values in Summary by % of null values
    summary = summary.sort_values(by='Null %', ascending=False).reset_index(drop=True)

    # Add dataset shape info above the table
    print(f"This dataset contain {df.shape[0]} rows")
    print(f"This dataset contain {df.shape[1]} columns")

    # Display the summary
    return summary

In [7]:
# ========================================================================================
# Taken from Module 3 - Final Project Milestone 1
# Link: https://github.com/LeeMcFarling/Module_3_Milestone_1/blob/main/Milestone_01.ipynb
# 
# Function purpose is to intake a variety of related columns (Foul 1, Foul 2, etc.) and create
# an indicator flag from it. i.e. 'Did penalty occur? (Y/N)'
# ========================================================================================


def create_indicator_from_columns(df, columns, new_column_name):
    # Initialize a boolean series with False for all rows.
    indicator = pd.Series(False, index=df.index)
    
    for col in columns:
        try:
            if col in df.columns:
                indicator = indicator | df[col].notnull()
            else:
                print(f"Warning: Column '{col}' not found. Skipping.")
        except Exception as e:
            print(f"Error processing column '{col}': {e}")
    
    # Assign the indicator as an integer column to the DataFrame
    df[new_column_name] = indicator.astype(int)
    return df

#### **DataFrame Functions**

In [8]:
# pd.set_option('display.max_columns', None) 
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None )


____

# **Big Data Bowl PreAnalysis**

In [9]:
BDB_All_Plays.head(1)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,Inj_Occured
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,0


Let's break the dataset down into numeric and categorical data

In [10]:
BDB_All_Plays_Numeric = numerify(BDB_All_Plays)
BDB_All_Plays_Categorical = categorify(BDB_All_Plays)

Here is a profile of the numeric data

In [11]:
profile_dataset(BDB_All_Plays_Numeric)

This dataset contain 8557 rows
This dataset contain 13 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,penaltyYards,Numeric,7801,91.17,756,60
1,defendersInBox,Numeric,7,0.08,8550,11
2,absoluteYardlineNumber,Numeric,1,0.01,8556,99
3,quarter,Numeric,0,0.00,8557,5
4,down,Numeric,0,0.00,8557,5
5,yardsToGo,Numeric,0,0.00,8557,32
6,yardlineNumber,Numeric,0,0.00,8557,50
7,preSnapHomeScore,Numeric,0,0.00,8557,42
8,preSnapVisitorScore,Numeric,0,0.00,8557,38
9,prePenaltyPlayResult,Numeric,0,0.00,8557,98


From looking at the data, it looks like we need to impute a 0 in the penaltyYards column if there was no injury, and either drop the rows. or inpute an unknown into the absoluteYardlineNumber and defendersInBox. 

In [12]:
profile_dataset(BDB_All_Plays_Categorical)

This dataset contain 8557 rows
This dataset contain 17 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,foulName3,Categorical,8556,99.99,1,1
1,foulName2,Categorical,8527,99.65,30,15
2,foulName1,Categorical,7821,91.40,736,29
3,dropBackType,Categorical,528,6.17,8029,8
4,yardlineSide,Categorical,125,1.46,8432,32
5,offenseFormation,Categorical,7,0.08,8550,7
6,personnelD,Categorical,1,0.01,8556,29
7,personnelO,Categorical,1,0.01,8556,30
8,gameId,Numeric,0,0.00,8557,122
9,pff_passCoverageType,Categorical,0,0.00,8557,3


There are a lot of null fields in the Foul / Penalty Fields. Let's roll that up into an indicator field to make things more simple to follow: 


- First, let's roll up Foul ID fields into the flag we need. 
- Second, let's consolidate Foul2 and Foul3

First, we'll use the fould ID fields to make a flag called 'foul_on_play'. 

In [13]:
columns = ['foulNFLId1', 'foulNFLId2', 'foulNFLId3']
new_column_name = 'foul_on_play'
BDB_All_Plays_Clean = create_indicator_from_columns(BDB_All_Plays, columns, new_column_name)

BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(columns=columns, index=1)

Next, because the prevalence of injuries is so small compared to the total number of plays overall, and because the number of plays in which both fouls and Injuries occured is smaller still, we should do some quick analysis to see if granular foul name type information is worth keeping, or if a simple foul flag will suffice. 

The reasoning is that if a certain foul only occured twice in the overall total sample, we cannot reliably identify patterns on such a small number of events. Keeping this level of dtail risks the temptation to bootstrap an exceedingly small sample, which could amplify incomplete or misleading interactions. within the dataset as a whole. 

In [14]:
print(f' Number of Plays in which Injuries & Fouls Occured: {len(BDB_All_Plays_Clean[(BDB_All_Plays_Clean['Inj_Occured'] == 1) & (BDB_All_Plays_Clean['foul_on_play'] == 1)])}')
print(f' Number of Plays in which Injuries Occured: {len(BDB_All_Plays_Clean[(BDB_All_Plays_Clean['Inj_Occured'] == 1)])}')
print(f' Number of Total Plays: {len(BDB_All_Plays)}')

 Number of Plays in which Injuries & Fouls Occured: 25
 Number of Plays in which Injuries Occured: 209
 Number of Total Plays: 8557


As suspected, the number of plays in which both fouls and injuries occured is 25 out of 8557, so ~ about a 0.0029 rate of incidence. 

Further investigating the data, we can see that there are 29 foul types in FoulName1 and 15 in FoulName2 and so on. 


Example: 

In [15]:
for col in ['foulName1', 'foulName2', 'foulName3']:
    print(f"\n--- {col} ---")
    print(f'{BDB_All_Plays_Clean[col].dropna().unique()}')


--- foulName1 ---
['Illegal Use of Hands' 'Taunting' 'Defensive Pass Interference'
 'Defensive Holding' 'Offensive Holding' 'Illegal Block Above the Waist'
 'Intentional Grounding' 'Offensive Pass Interference'
 'Unsportsmanlike Conduct' 'Defensive Offside' 'Illegal Formation'
 'Roughing the Passer' 'Unnecessary Roughness' 'Illegal Touch Pass'
 'Face Mask (15 Yards)' 'Ineligible Downfield Pass' 'Illegal Contact'
 'Disqualification' 'Illegal Blindside Block'
 'Lowering the Head to Initiate Contact' 'Chop Block' 'Low Block'
 'Illegal Shift' 'Tripping' 'Illegal Forward Pass' 'Illegal Substitution'
 'Illegal Motion' 'Horse Collar Tackle' 'Clipping']

--- foulName2 ---
['Unnecessary Roughness' 'Face Mask (15 Yards)' 'Tripping'
 'Defensive Offside' 'Roughing the Passer' 'Defensive Pass Interference'
 'Offensive Holding' 'Unsportsmanlike Conduct' 'Defensive Holding'
 'Illegal Use of Hands' 'Taunting' 'Disqualification'
 'Intentional Grounding' 'Offensive Pass Interference' 'Illegal Contact']

^ As such, the data is too granular on such a small scale to combine and then create dummy variables for > 29 different categories. Further investigations into foul types on injury rates are worth investigating but the sample size should be more than 8 weeks in one NFL season. 

For now, a foul_on_play flag will be deemed sufficient, and extra granular information will be removed to avoid any risks in confounding the model. 

In [16]:
drop_cols = ['foulName1', 'foulName2', 'foulName3']
BDB_All_Plays_Clean.drop(columns=drop_cols, axis=1, inplace=True)

In [17]:
profile_dataset(BDB_All_Plays_Clean)

This dataset contain 8556 rows
This dataset contain 28 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,penaltyYards,Numeric,7800,91.16,756,60
1,dropBackType,Categorical,528,6.17,8028,8
2,yardlineSide,Categorical,125,1.46,8431,32
3,defendersInBox,Numeric,7,0.08,8549,11
4,offenseFormation,Categorical,7,0.08,8549,7
5,personnelD,Categorical,1,0.01,8555,29
6,personnelO,Categorical,1,0.01,8555,30
7,absoluteYardlineNumber,Numeric,1,0.01,8555,99
8,prePenaltyPlayResult,Numeric,0,0.00,8556,98
9,Inj_Occured,Numeric,0,0.00,8556,2


Now let's check up on the features that still have null fields. 

In [18]:
for col in ['penaltyYards', 'dropBackType', 'yardlineSide', 'defendersInBox', 'offenseFormation']:
    print(f"\n--- {col} ---")
    print(f'{BDB_All_Plays_Clean[col].unique()}')
    print()


--- penaltyYards ---
[ nan   0.  14.   5.  26. -10.  16.  10.  -5. -14.   3.  19. -12.   8.
  15.  13.  11. -15.  17.  35.   2.   9.  -2.   6.  -4.  32.  21. -11.
  27.  36.   4.  12. -18.  25.  48.  -7.  28.  -3.  -6.  24.   7.   1.
  18.  43.  22.  23.  -9.  31.  20.  45.  47.  41.  33.  -8.  50.  39.
  46.  40.  38.  42. -13.]


--- dropBackType ---
['TRADITIONAL' 'SCRAMBLE_ROLLOUT_RIGHT' 'DESIGNED_ROLLOUT_RIGHT' nan
 'SCRAMBLE' 'DESIGNED_ROLLOUT_LEFT' 'UNKNOWN' 'DESIGNED_RUN'
 'SCRAMBLE_ROLLOUT_LEFT']


--- yardlineSide ---
['TB' 'DAL' nan 'ATL' 'PHI' 'PIT' 'BUF' 'NYJ' 'CAR' 'MIN' 'CIN' 'DET' 'SF'
 'HOU' 'JAX' 'IND' 'SEA' 'TEN' 'ARI' 'LAC' 'WAS' 'CLE' 'KC' 'MIA' 'NE'
 'NO' 'GB' 'NYG' 'DEN' 'CHI' 'LA' 'LV' 'BAL']


--- defendersInBox ---
[ 6.  7.  5.  4.  8.  3.  9. 10. nan 11.  2.  1.]


--- offenseFormation ---
['SHOTGUN' 'SINGLEBACK' 'EMPTY' 'I_FORM' 'JUMBO' 'PISTOL' nan 'WILDCAT']



Of the five features with missing values, three are easily imputed with 0 or unknown values -- the reasoning is as follows: 

- Penalty Yards: If no penalty, there are 0 penalty yards,
- drop back type:  An 'Unknown' type already exists. If N/A, then it's assumed unknown 
- YardLineSide: This is a string type categorical variable. It is easily imputed as NA with little change to the overall nature of the field. 

- OffensiveFormation: Again, as this is a string type categorical field, N/A values are easily imputed as UNKNONWN here. 

In [19]:
BDB_All_Plays_Clean = BDB_All_Plays_Clean.fillna({
    'penaltyYards': 0,
    'dropBackType': 'UNKNOWN',
    'yardlineSide': 'UNK',
    'offensiveFormation': 'UNKNOWN'
})

Because defenders in the box is not categorical, and is indeed numeric, adding an unknown category here would change the nature of the feature itself, additionally, we cannot impute '0' in this case because '0' is distinct from 'unknown' and doing so could confound the variable. 

To further investigate, let's query based on this field specifically

In [20]:
BDB_All_Plays_Clean[BDB_All_Plays_Clean['defendersInBox'].isna()].index

Index([916, 1570, 1654, 4887, 6874, 6899, 7912], dtype='int64')

Judging from the INJ_OCCURED field, there were no injuries that happened in this (extremely small) sample. In this case, it's safe just to drop the records and it will not have a significant impact on our target. 

In [21]:
BDB_dropable_records = BDB_All_Plays_Clean[BDB_All_Plays_Clean['defendersInBox'].isna()].index
BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(BDB_dropable_records)

And let's re-profile the dataset to see how  we're doing: 

In [22]:
profile_dataset(BDB_All_Plays_Clean)

This dataset contain 8549 rows
This dataset contain 28 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,gameId,Numeric,0,0.0,8549,122
1,playId,Numeric,0,0.0,8549,3761
2,Inj_Occured,Numeric,0,0.0,8549,2
3,pff_passCoverageType,Categorical,0,0.0,8549,3
4,pff_passCoverage,Categorical,0,0.0,8549,12
5,pff_playAction,Numeric,0,0.0,8549,2
6,dropBackType,Categorical,0,0.0,8549,8
7,personnelD,Categorical,0,0.0,8549,29
8,defendersInBox,Numeric,0,0.0,8549,11
9,personnelO,Categorical,0,0.0,8549,30


Great, we're almost ready to investigate the polynomial terms, all we have to do left is drop the ID type fields, make dummy variables for the categorical features, and then do some last minute checks to make sure we don't have perfect multicoliniarity. 

Regarding the ID fields:

In [23]:
ID_Fields = ['gameId', 'playId', 'playDescription']
BDB_All_Plays_Clean.drop(columns=ID_Fields, inplace=True)

And now let's handle the dummy variables: 


Ok so we are mostly good to go except for the gameClock variable which has 898 different unique variables. Let's change that into a floating point number instead of 14:58 etc, format it's in as nobody needs the negativity of 898 extra columns in a one-hot encoded variable in their lives. 

In [24]:
BDB_All_Plays_Clean['gameClock']

0       13:33
2       12:23
3       09:56
4       09:46
5       08:53
        ...  
8552    01:56
8553    01:07
8554    01:01
8555    00:39
8556    00:35
Name: gameClock, Length: 8549, dtype: object

In order to keep things consistant, let's convert this into what fraction of the quarter has elapsed. For instanct 7:30 out of a 15:00 game clock would be 0.5. 

In [25]:
minutes = BDB_All_Plays_Clean['gameClock'].str[:2].astype(int) # convert minutes to int
seconds = BDB_All_Plays_Clean['gameClock'].str[-2:].astype(int) # convert seconds to int

numerator = (minutes * 60 + seconds) # our data points in seconds
denominator = (60 * 15) # amount of seconds in 15 minutes

BDB_All_Plays_Clean['frac_quarter_elapsed'] = 1 - (numerator / denominator).round(2)


and let's check if it worked

In [26]:
BDB_All_Plays_Clean.head(1)

,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,...,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,Inj_Occured,foul_on_play,frac_quarter_elapsed
0,1,3,2,TB,DAL,TB,33,13:33,0,0,...,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,0,0,0.1


It worked. Now let's drop the gameClock variable. 

In [27]:
BDB_All_Plays_Clean.drop(columns='gameClock', axis=1, inplace=True)

In [28]:
BDB_All_Plays_Clean.head(5)

,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,passResult,...,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,Inj_Occured,foul_on_play,frac_quarter_elapsed
0,1,3,2,TB,DAL,TB,33,0,0,I,...,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,0,0,0.10
2,1,2,6,DAL,TB,DAL,34,0,0,C,...,"0 RB, 2 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0,Cover-3,Zone,0,0,0.17
3,1,1,10,DAL,TB,TB,39,0,0,I,...,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,1,Cover-3,Zone,0,0,0.34
4,1,3,15,DAL,TB,TB,44,0,0,I,...,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 4 LB, 4 DB",TRADITIONAL,0,Cover-3,Zone,0,0,0.35
5,1,2,5,TB,DAL,TB,11,0,0,C,...,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,0,0,0.41


next before one- hot encoding and doing VIF, let's make sure we know which of the columns to drop: 

In [30]:
BDB_All_Plays_Clean_Categorical = categorify(BDB_All_Plays_Clean)
profile_dataset(BDB_All_Plays_Clean_Categorical)

This dataset contain 8549 rows
This dataset contain 9 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,possessionTeam,Categorical,0,0.0,8549,32
1,defensiveTeam,Categorical,0,0.0,8549,32
2,yardlineSide,Categorical,0,0.0,8549,33
3,passResult,Categorical,0,0.0,8549,5
4,offenseFormation,Categorical,0,0.0,8549,7
5,personnelO,Categorical,0,0.0,8549,30
6,personnelD,Categorical,0,0.0,8549,29
7,dropBackType,Categorical,0,0.0,8549,8
8,pff_passCoverageType,Categorical,0,0.0,8549,3


Now let's make dummie variables: 

In [31]:
BDB_Dummies = pd.get_dummies(BDB_All_Plays_Clean_Categorical, drop_first=True)

print(BDB_Dummies.shape)

BDB_Dummies.head(1)

(8549, 170)


,possessionTeam_ATL,possessionTeam_BAL,possessionTeam_BUF,possessionTeam_CAR,possessionTeam_CHI,possessionTeam_CIN,possessionTeam_CLE,possessionTeam_DAL,possessionTeam_DEN,possessionTeam_DET,...,"personnelD_6 DL, 4 LB, 1 DB",dropBackType_DESIGNED_ROLLOUT_RIGHT,dropBackType_DESIGNED_RUN,dropBackType_SCRAMBLE,dropBackType_SCRAMBLE_ROLLOUT_LEFT,dropBackType_SCRAMBLE_ROLLOUT_RIGHT,dropBackType_TRADITIONAL,dropBackType_UNKNOWN,pff_passCoverageType_Other,pff_passCoverageType_Zone
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


And there were some issues, so let's cast the boolean columns as integers. 

In [32]:
bool_cols = BDB_Dummies.select_dtypes(include='bool').columns
BDB_Dummies[bool_cols] = BDB_Dummies[bool_cols].astype(int)

and then let's concatenate the dummie variables created with the numeric features. 

In [33]:
BDB_All_Plays_Clean_Numeric = numerify(BDB_All_Plays_Clean)
BDB_All_Plays_Model_Ready = pd.concat([BDB_All_Plays_Clean_Numeric, BDB_Dummies], axis=1)

print(BDB_All_Plays_Model_Ready.shape)
BDB_All_Plays_Model_Ready.head(5)

(8549, 185)


,quarter,down,yardsToGo,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,penaltyYards,prePenaltyPlayResult,playResult,absoluteYardlineNumber,...,"personnelD_6 DL, 4 LB, 1 DB",dropBackType_DESIGNED_ROLLOUT_RIGHT,dropBackType_DESIGNED_RUN,dropBackType_SCRAMBLE,dropBackType_SCRAMBLE_ROLLOUT_LEFT,dropBackType_SCRAMBLE_ROLLOUT_RIGHT,dropBackType_TRADITIONAL,dropBackType_UNKNOWN,pff_passCoverageType_Other,pff_passCoverageType_Zone
0,1,3,2,33,0,0,0.0,0,0,43.0,...,0,0,0,0,0,0,1,0,0,0
2,1,2,6,34,0,0,0.0,5,5,76.0,...,0,0,0,0,0,0,1,0,0,1
3,1,1,10,39,0,0,0.0,0,0,49.0,...,0,0,0,0,0,0,1,0,0,1
4,1,3,15,44,0,0,0.0,0,0,54.0,...,0,0,0,0,0,0,1,0,0,1
5,1,2,5,11,0,0,0.0,10,10,21.0,...,0,0,0,0,0,0,1,0,0,0


___

# **Big Data Bowl: Variance Inflation Factor**

Next, we need to check the model for interactions between the polynomial terms. We will use the VIF method as detailed at the link below. 

#### **VIF Interpretation**

- Values near 1 mean that the features are independent
- Values between 1 and 5 shows moderate correlation
- Values > 10 show problematic levels of multicolinearity


Source: https://www.geeksforgeeks.org/python/detecting-multicollinearity-with-vif-python/

In [108]:
def VIF_Analyze(df):
    df_ready = df.select_dtypes(include=[np.number, 'bool', 'boolean'])
    df_not_ready = df.select_dtypes(exclude=[np.number])

    VIF_data = pd.DataFrame()
    VIF_data['feature'] = df_ready.columns
    VIF_data['VIF'] = [variance_inflation_factor(df_ready.values, i) for i in range(len(df_ready.columns))]

    print(f'These Columns were not formatted correctly. Could not include in analysis \n {df_not_ready.columns}')
    return VIF_data

In [35]:
VIF_data = VIF_Analyze(BDB_All_Plays_Model_Ready)
VIF_data.head(5)

/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


These Columns were not formatted correctly. Could not include in analysis 
 Index([], dtype='object')


,feature,VIF
0,quarter,24.520859
1,down,9.297846
2,yardsToGo,7.675563
3,yardlineNumber,8.629431
4,preSnapHomeScore,6.244800


#### **Notes on the following VIF Analysis**

(I put the notes up here because I figured it would be more intuitive than scrolling to the bottom of a big data frame.)

So it looks like pff pass coverage and offensive formations are exhibiting perfect multicolliniarity, even after we drop one of the dummie variables. My guess would be that personnel O (offensive formation) and personell O (the specific number of offensive personel on the field.) are perfectly coordinated -- Intuitively, this makes sense, and might even be reflected in the Defensive formations.


So we'll start by dropping PFF Pass Coverage columns and then we'll re-run VIF and go from there. 

In [36]:
VIF_data[VIF_data['VIF'] > 5].sort_values(by='VIF', ascending=False)

,feature,VIF
175,"personnelD_6 DL, 4 LB, 1 DB",inf
127,"personnelO_1 RB, 4 TE, 0 WR",inf
124,"personnelO_1 RB, 1 TE, 3 WR",345.167874
163,"personnelD_4 DL, 2 LB, 5 DB",193.587726
155,"personnelD_2 DL, 4 LB, 5 DB",118.747272
125,"personnelO_1 RB, 2 TE, 2 WR",91.813490
159,"personnelD_3 DL, 3 LB, 5 DB",89.919853
10,defendersInBox,76.507036
154,"personnelD_2 DL, 3 LB, 6 DB",61.867782
164,"personnelD_4 DL, 3 LB, 4 DB",56.374299


### Whole Columns that were dropped

In [63]:
# BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(columns='pff_passCoverage', axis=1) 

# Added after VIF a second time
BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(columns='personnelO', axis=1) 

# Added after VIF a third time
BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(columns='personnelD', axis=1) 

# Added after VIF a fourth time
BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(columns='prePenaltyPlayResult', axis=1) # similar with playResult

# Added after VIF a fifth time
BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(columns='defendersInBox', axis=1) # Not correlated highly with target, and high VIF

# Dropped after VIF a sixth time
BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(columns='quarter', axis=1) # Not highly correlated with target, high VIF

#### One-Hot Dummie Columns that were Dropped

For the following, only one one-hot column was dropped (as opposed to the whole thing) because this feature was highly correlated to the target. 

In [38]:
BDB_All_Plays_Model_Ready = BDB_All_Plays_Model_Ready.drop(columns='dropBackType_TRADITIONAL', axis=1) # Dropping only this one as feature is highly correlated with target

#### BDB Logistic Regression Dataset

In [39]:
BDB_All_Plays_Model_Ready.to_csv('../../Dataset Analysis/BDB_All_Plays_Model_Ready.csv', index=False)

____

## **Dataset 2: First and Future**


As a refresher, this dataset is from an NFL Kaggle comepetition looking at the effect different factors like field type and weather have on lower extremity injuries (ankles, feet, knee, etc.)

#### **Pre Analysis**

In [40]:
profile_dataset(FNF_All_Plays)

This dataset contain 267006 rows
This dataset contain 22 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,Surface,Categorical,266929,99.97,77,2
1,InjuryLength,Categorical,266929,99.97,77,4
2,BodyPart,Categorical,266929,99.97,77,3
3,time,Numeric,45,0.02,266961,5872
4,distance,Numeric,45,0.02,266961,219352
5,speed,Numeric,45,0.02,266961,259649
6,direction,Numeric,45,0.02,266961,266790
7,y,Numeric,45,0.02,266961,266183
8,x,Numeric,45,0.02,266961,266738
9,PlayKey,Categorical,0,0.00,267006,267005


First let's drop high - null count columns:

In [ ]:
FNF_All_Plays.drop(columns=['Surface', 'InjuryLength', 'BodyPart', 'PlayerKey', 'PlayKey', 'PositionGroup'], inplace=True)

For those with a low number of nulls, we can impute with the mean and median (depending on which works better for that column)

In [98]:
FNF_All_Plays.describe()

,PlayerDay,PlayerGame,Temperature,PlayerGamePlay,x,y,direction,speed,distance,time,Inj_Occured
count,267006.00000,267006.000000,267006.000000,267006.000000,267006.000000,267006.000000,267006.000000,267006.000000,267006.000000,267006.000000,267006.000000
mean,210.45084,13.799117,60.924500,29.058605,60.166875,26.750706,179.783725,1.327122,0.137821,14.265425,0.000288
std,183.64350,8.342881,16.761277,19.626526,24.169652,7.333594,33.945712,0.730692,0.072958,6.045155,0.016979
min,-62.00000,1.000000,9.000000,1.000000,1.399103,-2.746723,27.642222,0.000000,0.000000,0.250000,0.000000
25%,43.00000,7.000000,49.000000,13.000000,41.300024,22.261076,157.413436,0.821305,0.087472,10.800000,0.000000
50%,102.00000,13.000000,61.000000,26.000000,60.264622,26.755823,179.781865,1.198884,0.124811,13.600000,0.000000
75%,400.00000,20.000000,72.000000,43.000000,79.019011,31.234398,202.190209,1.661803,0.170848,16.500000,0.000000
max,480.00000,32.000000,97.000000,102.000000,123.406408,56.803038,329.506753,9.059091,0.914848,93.450000,1.000000


The x column is the only one with a high enough std to be suspicious, but when I checked the median it was similar. 

In [99]:
FNF_All_Plays = FNF_All_Plays.fillna({
    'time': 14.26,
    'distance': 0.13,
    'speed': 1.32, 
    'direction': 170.78,
    'y': 26.75,
    'x': 60.166
})

In [100]:
profile_dataset(FNF_All_Plays)

This dataset contain 267006 rows
This dataset contain 16 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,PlayerDay,Numeric,0,0.0,267006,215
1,PlayerGame,Numeric,0,0.0,267006,32
2,StadiumType,Categorical,0,0.0,267006,3
3,FieldType,Categorical,0,0.0,267006,2
4,Temperature,Numeric,0,0.0,267006,78
5,Weather,Categorical,0,0.0,267006,8
6,PlayType,Categorical,0,0.0,267006,7
7,PlayerGamePlay,Numeric,0,0.0,267006,102
8,Position,Categorical,0,0.0,267006,23
9,x,Numeric,0,0.0,267006,266738


In [101]:
FNF_All_Plays_Categorical = categorify(FNF_All_Plays)
FNF_All_Plays_Numeric = numerify(FNF_All_Plays)

In [102]:
profile_dataset(FNF_All_Plays_Categorical)

This dataset contain 267006 rows
This dataset contain 5 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,StadiumType,Categorical,0,0.0,267006,3
1,FieldType,Categorical,0,0.0,267006,2
2,Weather,Categorical,0,0.0,267006,8
3,PlayType,Categorical,0,0.0,267006,7
4,Position,Categorical,0,0.0,267006,23


In [103]:
FNF_Dummies = pd.get_dummies(FNF_All_Plays_Categorical, drop_first=True)

print(FNF_Dummies.shape)

FNF_Dummies.head(1)

(267006, 38)


,StadiumType_Outdoors,StadiumType_Unknown,FieldType_Synthetic,Weather_Fog,Weather_N/A (Indoors),Weather_Partly Cloudy,Weather_Rain,Weather_Snow,Weather_Sunny,Weather_Unknown,...,Position_NT,Position_OLB,Position_P,Position_QB,Position_RB,Position_S,Position_SS,Position_T,Position_TE,Position_WR
0,True,False,True,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False


In [104]:
FNF_bool_cols = FNF_Dummies.select_dtypes(include='bool').columns
FNF_Dummies[FNF_bool_cols] = FNF_Dummies[FNF_bool_cols].astype(int)

In [105]:
FNF_Model_Ready = pd.concat([FNF_Dummies, FNF_All_Plays_Numeric], axis=1)

print(FNF_Model_Ready.shape)
FNF_Model_Ready.head(5)

(267006, 49)


,StadiumType_Outdoors,StadiumType_Unknown,FieldType_Synthetic,Weather_Fog,Weather_N/A (Indoors),Weather_Partly Cloudy,Weather_Rain,Weather_Snow,Weather_Sunny,Weather_Unknown,...,PlayerGame,Temperature,PlayerGamePlay,x,y,direction,speed,distance,time,Inj_Occured
0,1,0,1,0,0,0,0,0,1,0,...,1,63,1,87.665753,28.221104,186.148361,0.535753,0.056288,14.90,0
1,1,0,1,0,0,0,0,0,1,0,...,1,63,2,86.616462,29.144077,211.949423,0.849692,0.090077,12.95,0
2,1,0,1,0,0,0,0,0,1,0,...,1,63,3,79.677264,28.900203,194.134122,0.331014,0.036081,14.75,0
3,1,0,1,0,0,0,0,0,1,0,...,1,63,4,72.469449,26.680157,163.762205,0.523701,0.054016,6.30,0
4,1,0,1,0,0,0,0,0,1,0,...,1,63,5,65.407846,24.865577,222.120577,0.970385,0.099769,12.95,0


In [115]:
FNF_Model_Ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267006 entries, 0 to 267005
Data columns (total 47 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   StadiumType_Outdoors   267006 non-null  int64  
 1   StadiumType_Unknown    267006 non-null  int64  
 2   FieldType_Synthetic    267006 non-null  int64  
 3   Weather_Fog            267006 non-null  int64  
 4   Weather_N/A (Indoors)  267006 non-null  int64  
 5   Weather_Partly Cloudy  267006 non-null  int64  
 6   Weather_Rain           267006 non-null  int64  
 7   Weather_Snow           267006 non-null  int64  
 8   Weather_Sunny          267006 non-null  int64  
 9   Weather_Unknown        267006 non-null  int64  
 10  PlayType_Field Goal    267006 non-null  int64  
 11  PlayType_Kickoff       267006 non-null  int64  
 12  PlayType_Pass          267006 non-null  int64  
 13  PlayType_Punt          267006 non-null  int64  
 14  PlayType_Rush          267006 non-nu

# **First and Future: Variance Inflation Factor**

Now, just like the Big Data Bowl, we will check the interactions between the polynomial terms using VIF. 


#### **VIF Interpretation**

- Values near 1 mean that the features are independent
- Values between 1 and 5 shows moderate correlation
- Values > 10 show problematic levels of multicolinearity

In [121]:
FNF_VIF_Data = VIF_Analyze(FNF_Model_Ready)
FNF_VIF_Data[FNF_VIF_Data['VIF'] > 10].sort_values(by='VIF', ascending=False)

These Columns were not formatted correctly. Could not include in analysis 
 Index([], dtype='object')


,feature,VIF
12,PlayType_Pass,17.230550
38,Temperature,13.791059
41,y,12.858468
14,PlayType_Rush,11.259430


In [119]:
FNF_Model_Ready.columns

Index(['StadiumType_Outdoors', 'StadiumType_Unknown', 'FieldType_Synthetic',
       'Weather_Fog', 'Weather_N/A (Indoors)', 'Weather_Partly Cloudy',
       'Weather_Rain', 'Weather_Snow', 'Weather_Sunny', 'Weather_Unknown',
       'PlayType_Field Goal', 'PlayType_Kickoff', 'PlayType_Pass',
       'PlayType_Punt', 'PlayType_Rush', 'PlayType_Unknown', 'Position_CB',
       'Position_DB', 'Position_DE', 'Position_DT', 'Position_FS',
       'Position_G', 'Position_HB', 'Position_ILB', 'Position_K',
       'Position_LB', 'Position_MLB', 'Position_Missing Data', 'Position_NT',
       'Position_OLB', 'Position_P', 'Position_QB', 'Position_RB',
       'Position_S', 'Position_SS', 'Position_T', 'Position_TE', 'Position_WR',
       'PlayerDay', 'Temperature', 'PlayerGamePlay', 'x', 'y', 'speed', 'time',
       'Inj_Occured'],
      dtype='object')

Now I took turns runnging the cell above, and then taking out the most egregious offenders in terms of highest inflation factors, (being multicolinear with other factors) and then re-running the analysis. 

In [ ]:
# Run 1: 
# FNF_Model_Ready.drop(columns=['distance'], inplace=True) # Extremely high VIF (over 1,000) - Didn't have strong correlation with Target

# Run 2: 
# FNF_Model_Ready.drop(columns=['PlayerGame'], inplace=True) # High VIF (>20) Was highly correlated with playerDay

# Run 3: 
# FNF_Model_Ready.drop(columns=['direction'], inplace=True) # High VIF (24.5), not strongly correlated with Target Variable

# Run 4: 
# FNF_Model_Ready.drop(columns=['PlayType_Unknown'], inplace=True) # Attempt to solve high VIFs of other playtypes that were more correlated with target

In [122]:
FNF_VIF_Data = VIF_Analyze(FNF_Model_Ready)
FNF_VIF_Data[FNF_VIF_Data['VIF'] > 5].sort_values(by='VIF', ascending=False)

These Columns were not formatted correctly. Could not include in analysis 
 Index([], dtype='object')


,feature,VIF
12,PlayType_Pass,17.230550
38,Temperature,13.791059
41,y,12.858468
14,PlayType_Rush,11.259430
42,speed,7.880362
0,StadiumType_Outdoors,7.583282
43,time,6.967598
40,x,6.857383


So of those that remain, the playtype_pass and playtype_rush have a high VIF, this makes a lot of sense because these two are inversely correlated (most plays in the NFL are either a pass or a rush.) Temperature and Stadium Type also seem to have a high VIF as well, which also makes sense, as outdoor stadiums can get very hot or very cold. Finally player positioning on the field, (in terms of X and Y) have a high VIF, which again makes sense. 


I'm not going to drop any more columns here, because even though these variables seem to have a very strong correlation to each other, they also have the highest predictive power of the features that remain. Even though there is multicollinearity in this dataset, I suspect that it can be managed properly by using tree-based models (like Random Forest, etc.) instead of the base-logistic regression. 

So for now, we will keep the remaining features as they are, and we will export the DF as it is for future weeks. 

In [123]:
FNF_Model_Ready.to_csv('../../Dataset Analysis/FNF_Model_Ready.csv', index=False)

_____

# **Punt Data Analytics:**

As a refresher, this dataset was meant to analyze head injuries during punt plays. 

#### **Pre-Analysis**

First, let's check up on the features

In [125]:
profile_dataset(PDA_All_Plays)

This dataset contain 6681 rows
This dataset contain 25 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,Temperature,Numeric,586,8.77,6095,85
1,StadiumType,Categorical,381,5.70,6300,33
2,Turf,Categorical,9,0.13,6672,21
3,GameWeather,Categorical,0,0.00,6681,9
4,PlayDescription,Categorical,0,0.00,6681,6681
5,Score_Home_Visiting,Categorical,0,0.00,6681,603
6,Poss_Team,Categorical,0,0.00,6681,33
7,Play_Type,Categorical,0,0.00,6681,1
8,Quarter,Numeric,0,0.00,6681,5
9,YardLine,Categorical,0,0.00,6681,1470


In [131]:
PDA_Categorical = categorify(PDA_All_Plays)
profile_dataset(PDA_Categorical)

This dataset contain 6681 rows
This dataset contain 15 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,StadiumType,Categorical,381,5.70,6300,33
1,Turf,Categorical,9,0.13,6672,21
2,Season_Type,Categorical,0,0.00,6681,3
3,Game_Date,Categorical,0,0.00,6681,142
4,Game_Day,Categorical,0,0.00,6681,6
5,Game_Site,Categorical,0,0.00,6681,39
6,Start_Time,Categorical,0,0.00,6681,9
7,Home_Team,Categorical,0,0.00,6681,33
8,Visit_Team,Categorical,0,0.00,6681,33
9,GameWeather,Categorical,0,0.00,6681,9


Alright, PlayDescription can be dropped (unique for each play), as well as PlayID.

Weather and Outdoor weather, as well as Stadium and Game_site are redundant, so we'll only keep the best one of those. 

Additionally, there are some columns like Score_Home_Visiting, YardLine, Game_Clock, Game_Date, Start_Time should be formatted to be numerical. 




First we'll drop the PlayDescription and PlayID variables. 

In [128]:
PDA_All_Plays.drop(columns=['PlayID', 'PlayDescription'], inplace=True)

And we'll check the redundant columns: 

In [129]:
for col in ['GameWeather', 'OutdoorWeather', 'Stadium', 'Game_Site']:
    print(f"\n--- {col} ---")
    print(f'{PDA_All_Plays[col].unique()}')
    print()


--- GameWeather ---
['Sunny' 'Partly Cloudy' 'Unknown' 'Rain' 'Cloudy' 'Hazy' 'N/A (Indoors)'
 'Snow' 'Fog']


--- OutdoorWeather ---
['Sunny' 'Partly Cloudy' 'Unknown' 'Rain'
 'T-Storms increasing over 30 minutes before kickoff' 'Cloudy' 'Hazy'
 'Hot' 'Cloudy, 74 degr. Wind SSW at 13 mph '
 '82 deg F, Winds W at 7mph, Humidity 57%'
 'Chance of Showers & Thunderstorms' 'Mostly Clear.'
 'Slight Chance of Rain' 'Sunny Skies' 'N/A (Indoors)' 'Mostly cloudy'
 '20% Chance of Rain' 'Falling to Upper 60s' 'Clear'
 'Partly sunny, 83 degr. Wind SSE at 7 mph' 'Overcast, muggy'
 'Party Cloudy' '72 deg F, Winds NNE 12 mph, Humidity 59%'
 'T: 82; H: 66%; W: ENE: 21' 'Falling to lows 70s' '10% Chance of Rain'
 '79' 'T: 84; H: 35%; W: NW 2 mph'
 'Partly sunny, 76 degr. Wind S at 5 mph'
 'Forecast: Temps in low 60s, clear skies, light winds'
 '73 deg F, Winds SW at 4mph, Humidy 72%'
 'Forecast: plentiful sunshine, high temp 67, winds 5-10 mph'
 '40% Chance of Rain' 'Cloudy, 85 degr. Wind SSW@ 8 mph'


After checking my EDA for semester 2, I formatted and condensed the weather and Game_Site columns, but not the outdoor weather column and Stadium columns. So we'll drop the two that haven't been formatted. 

In [130]:
PDA_All_Plays.drop(columns=['Stadium', 'OutdoorWeather'], inplace=True)

And now let's check the numeric data that is showing up as categorical. 

Feature	Type	Null Values	Null %	Count (Non-Null)	Unique Values
0	StadiumType	Categorical	381	5.70	6300	33
1	Turf	Categorical	9	0.13	6672	21
2	Season_Type	Categorical	0	0.00	6681	3
3	Game_Date	Categorical	0	0.00	6681	142
4	Game_Day	Categorical	0	0.00	6681	6
5	Game_Site	Categorical	0	0.00	6681	39
6	Start_Time	Categorical	0	0.00	6681	9
7	Home_Team	Categorical	0	0.00	6681	33
8	Visit_Team	Categorical	0	0.00	6681	33
9	GameWeather	Categorical	0	0.00	6681	9
10	Game_Clock	Categorical	0	0.00	6681	891
11	YardLine	Categorical	0	0.00	6681	1470
12	Play_Type	Categorical	0	0.00	6681	1
13	Poss_Team	Categorical	0	0.00	6681	33
14	Score_Home_Visiting	Categorical	0	0.00	6681	603

In [142]:
for col in [
    # 'Score_Home_Visiting',
    'YardLine', 
    'Game_Clock', 
    'Game_Date', 
    'StadiumType'
 ]:
    print(f"\n--- {col} ---")
    print(f'{PDA_All_Plays[col].unique()}')
    print()


--- YardLine ---
['LA 47' 'LA 29' 'DAL 18' ... 'KC 5' 'ATL 12' 'ATL 11']


--- Game_Clock ---
['12:30' '12:08' '10:01' '00:21' '10:26' '09:00' '00:27' '13:30' '05:28'
 '06:44' '05:50' '02:54' '01:51' '06:10' '01:59' '09:32' '08:37' '01:53'
 '14:18' '04:36' '02:48' '05:00' '03:29' '08:38' '06:06' '02:26' '06:59'
 '03:23' '00:06' '11:40' '10:17' '01:57' '13:24' '11:14' '03:44' '00:16'
 '12:19' '06:39' '03:40' '13:00' '05:16' '05:07' '01:07' '15:00' '11:37'
 '09:04' '12:04' '03:57' '02:12' '00:14' '14:19' '12:44' '05:08' '02:57'
 '01:09' '00:43' '13:38' '12:07' '05:56' '02:42' '00:18' '13:27' '03:51'
 '03:24' '09:16' '08:36' '00:35' '14:13' '12:05' '11:04' '09:01' '05:12'
 '04:10' '01:01' '09:20' '08:11' '04:23' '05:19' '11:01' '09:58' '08:24'
 '04:05' '03:05' '08:25' '03:55' '02:55' '02:24' '01:48' '08:16' '06:14'
 '04:37' '14:04' '12:02' '10:38' '10:23' '08:40' '00:38' '12:46' '06:30'
 '03:35' '09:59' '04:34' '02:38' '02:27' '07:42' '12:34' '08:07' '03:09'
 '02:00' '11:42' '03:33' '13:

First, let's split the home - away score into two columns: 
- one for the home score
- And a second for away score


In [ ]:
PDA_All_Plays['home_score'] = PDA_All_Plays['Score_Home_Visiting'].str[-2:].astype(int)
PDA_All_Plays['away_score'] = PDA_All_Plays['Score_Home_Visiting'].str[:2].astype(int)

And we'll drop the first column

In [ ]:
PDA_All_Plays.drop(columns=['Score_Home_Visiting'], inplace=True)

Next, we'll format the stadium type the way it was formatted in First and Future. 

In [141]:
stadium_mapping = {
    "Outdoor": "Outdoors",
    "Oudoor": "Outdoors",
    "Outdoors": "Outdoors",
    "Open": "Outdoors",
    "Outdoor Retr Roof": "Outdoors",
    "Oudoor": "Outdoors",
    "Outddors": "Outdoors",
    "Retr. Roof-Open": "Outdoors",
    "Retr. Roof - Open": "Outdoors",
    "Outdor": "Outdoors",
    "Outside": "Outdoors",
    "Heinz Field": "Outdoors",
    "Cloudy": "Outdoors",
    "Outdoor Retr Roof-Open" : "Outdoors", 
    "Ourdoor" : "Outdoors",
    "Indoor, Open Roof" : "Outdoors",
    "outdoor" : "Outdoors",
    "Outdoors ": "Outdoors",

    # "Indoors": "Indoors",
    "Closed Dome": "Indoors",
    "Domed": "Indoors",
    "closed": "Indoors",
    "Dome": "Indoors",
    "Indoor": "Indoors",
    "Retr. Roof-Closed": "Indoors",
    "Retr. Roof - Closed": "Indoors",
    "Retractable Roof": "Indoors",
    "Domed, Open": "Indoors",
    "Domed, open": "Indoors",
    "Roof Closed": "Indoors",
    "Dome, closed": "Indoors",
    "Domed, closed" : "Indoors",
    "Indoor, Roof Closed": "Indoors", 
    "Retr. Roof Closed": "Indoors",
    "Retr. roof - closed": "Indoors",
    "Non-Retractable Dome": "Indoors",
    "Indoor, non-retractable roof": "Indoors",
    "Indooor": "Indoors",
    "Indoor, fixed roof": "Indoors",
    "Indoor, Non-Retractable Dome": "Indoors",
    "Indoor, Fixed Roof": "Indoors",
    "Indoors (Domed)": "Indoors",

    "nan": "Unknown",
    "Bowl": "Unknown",
    "Turf": "Unknown"
}

PDA_All_Plays['StadiumType'] = PDA_All_Plays['StadiumType'].replace(stadium_mapping)
PDA_All_Plays['StadiumType'] = PDA_All_Plays['StadiumType'].replace(stadium_mapping).fillna("Unknown")

print(PDA_All_Plays['StadiumType'].unique())

['Outdoors' 'Unknown' 'Indoors']


And we can format the game_clock field, the same way that we formatted the field in our Big Data Bowl dataset earlier in this analysis. 

In [143]:
minutes = PDA_All_Plays['Game_Clock'].str[:2].astype(int) # convert minutes to int
seconds = PDA_All_Plays['Game_Clock'].str[-2:].astype(int) # convert seconds to int

numerator = (minutes * 60 + seconds) # our data points in seconds
denominator = (60 * 15) # amount of seconds in 15 minutes

PDA_All_Plays['frac_quarter_elapsed'] = 1 - (numerator / denominator).round(2)

And then we'll drop the original gameclock field. 

In [144]:
PDA_All_Plays.drop(columns=['Game_Clock'], inplace=True)

Format the Game_Date field as just a date, and not datet